**Navigation**
1. [Dependencies](#Dependencies)
2. [Model Loading and Local Inference](#Model-Loading-and-Local-Inference)

# **Dependencies**

In [1]:
!pip install datasets transformers torch

from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

langs = ['java', 'python', 'pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# **Model Loading and Local Inference**

In [2]:
samples = {
    "java": [
        "this is the main method that starts the java application | main.java",
        "a method to calculate the factorial of a number | factorial.java",
        "a class that represents a bank account with deposit and withdrawal methods | bankaccount.java",
        "a simple loop to print numbers from 1 to 10 | loopexample.java",
        "a method that checks whether a number is prime | primechecker.java"
    ],
    "python": [
        "a function that returns the fibonacci sequence up to a given number | fibonacci.py",
        "this code reads a text file and prints its contents | file_reader.py",
        "a class that implements a simple stack data structure | stack.py",
        "this script generates random numbers and calculates their average | random_average.py",
        "a function that takes two arguments and returns their sum | sum_two_numbers.py"
    ],
    "pharo": [
        "this class defines a simple bank account with deposit and withdrawal functionality | bankaccount.pharo",
        "a loop that prints numbers from 1 to 5 | loop.pharo",
        "a method that calculates the area of a circle given the radius | circle_area.pharo",
        "a script that prints 'hello, pharo!' to the transcript | hello.pharo",
        "this code defines a person class with name and age attributes | person.pharo"
    ]
}

def inference(lang, input_samples):
    model_name = f"harisathar04/graphic-nlbse-{lang}"
    model = AutoModelForSequenceClassification.from_pretrained(model_name, use_auth_token=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    inputs = tokenizer(input_samples, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = torch.sigmoid(outputs.logits)
        predictions = (logits > 0.5).int()

    return predictions.cpu().numpy()

for lang, input_samples in samples.items():
    print(f"Performing inference for {lang}...")

    predictions = inference(lang, input_samples)

    for i, (sample, prediction) in enumerate(zip(input_samples, predictions)):
        print(f"Sample {i + 1} ({lang}):")
        print(f"Comment: {sample}\nPrediction: {prediction}\n")

Performing inference for java...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Sample 1 (pharo):
Comment: this class defines a simple bank account with deposit and withdrawal functionality | bankaccount.pharo
Prediction: [0 0 1 0 0 0 0]

Sample 2 (pharo):
Comment: a loop that prints numbers from 1 to 5 | loop.pharo
Prediction: [0 0 1 0 1 0 0]

Sample 3 (pharo):
Comment: a method that calculates the area of a circle given the radius | circle_area.pharo
Prediction: [0 0 1 0 0 0 0]

Sample 4 (pharo):
Comment: a script that prints 'hello, pharo!' to the transcript | hello.pharo
Prediction: [0 0 1 0 1 0 0]

Sample 5 (pharo):
Comment: this code defines a person class with name and age attributes | person.pharo
Prediction: [0 0 0 0 0 1 0]

